In [13]:
import os, sys, inspect
import csv
import json

import math
import numpy as np
import pandas as pd


import matplotlib
from matplotlib import pyplot as plt

import collections
import time
import datetime

import re
import hashlib

import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException

working_dir = os.path.dirname(
    os.path.abspath(
        inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(working_dir)
sys.path.insert(0, parent_dir)

from scripts_python import scraper as scr

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1. Chrome - Help - About Google Chrome
2. check version; e.g. `Version 79.0.3945.117 (Official Build) (64-bit)`
3. [download relevant ChromeDriver](https://chromedriver.chromium.org/downloads)

In [2]:
# Get credentials.
with open('../ignorabilia/glassdoor_scraper.csv', mode='r') as f_in:
    credentials = {k: v for k, v in csv.reader(f_in)}

In [3]:
# Open browser.
browser = scr.get_web_browser()
browser.implicitly_wait(10)
scr.load_main_page(browser=browser)

# Accept cookies.
scr.accept_cookies(browser=browser)

# Signt in to glassdoor.
scr.sign_in(
    browser=browser,
    e_mail=credentials['e_mail'],
    password=credentials['password'])

https://www.glassdoor.co.uk


In [4]:
# Run the search for particular keywords and location.
scr.do_search(
        browser=browser,
        keywords='data scientist',
        location='Paris (France)')

In [5]:
# Adjust filters, including jobtype, post_age, and radius (from location centre).
scr.set_filters(
        browser=browser,
        jobtype='fulltime',
        post_age='7',
        radius='5',
        )

In [6]:
scraped_results = []
search_date = datetime.datetime.now()

xpath = (
    f"//div[@id='PageContent']"
    f"//div[@id='JobResults']"
    f"//p[@class='jobsCount']"
    )
element = scr.get_element_when_present(browser=browser, xpath=xpath)

n_jobs = int(element.text.split()[0])
n_pages = math.ceil(346 / 30)

print(
    f"\tjobs {n_jobs}\n"
    f"\tpages {n_pages}")

	jobs 437
	pages 12


In [ ]:
for p in range(n_pages-9):
    
    xpath = (
        f"//div[@id='PageContent']"
        f"//div[@id='JobResults']"
        f"//ul[@class='jlGrid hover']"
        f"//li[@class='jl']")
#     Wait for all element to be located.
    WebDriverWait(
        browser, 10).until(
            EC.visibility_of_all_elements_located(
                (By.XPATH, xpath)))
#     Get all elements.
    jobs_on_page = browser.find_elements_by_xpath(xpath)
    
    for j in jobs_on_page[:5]:
        j.click()
        browser.implicitly_wait(5)
        
        name, rating = scr.get_company_name_and_rating(browser=browser)
        browser.implicitly_wait(5)
        
        job_title, job_description = scr.get_job_title_and_description(browser=browser)
        job_description_encoded = job_description.encode('utf-8')
        job_description_hash = hashlib.sha224(job_description_encoded).hexdigest()

        try:
            size, url = scr.get_company_size_and_url(browser=browser)
        except NoSuchElementException:
            size = 'NA'
            url = 'NA'
            
        scraped_results.append({
            'search_date': str(search_date),
            'company_name': name,
            'company_rating': rating,
            'company_size': size,
            'company_url': url,
            'job_title': job_title,
            'job_description': job_description,
            'job_hash': job_description_hash})
        
    scr.switch_to_next_page(browser=browser)

In [8]:
browser.quit()

In [47]:
def append_to_file(
    data,
    file_path):
    
    with open(file_path, 'a+') as file:
        file.write(f'{json.dumps(data)}\n')

In [53]:
for job in scraped_results:
#     json_job = json.dumps(job)
    append_to_file(job,
    file_path='../test.txt')

In [36]:
def get_saved_jobs(
    file_path):
    with open(file_path, 'r') as json_file:
        lines = json_file.readlines()
    jobs = []
    for l in lines:
        jobs.append(json.loads(l))
    jobs_df = pd.DataFrame(jobs)
    return jobs_df